In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
from collections import Counter
import numpy as np
from sklearn.metrics import f1_score
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    precision_recall_curve,
    average_precision_score
)
from sklearn.preprocessing import label_binarize


In [24]:
IMG_SIZE = 224

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),  # was too high
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [25]:
dataset = ImageFolder(r"E:\EESAWP\train")
class_names = dataset.classes
num_classes = len(class_names)


In [26]:
TEST_RATIO = 0.2

test_size = int(TEST_RATIO * len(dataset))
train_size = len(dataset) - test_size
torch.manual_seed(42)


train_ds, test_ds = random_split(dataset, [train_size, test_size])

In [27]:
train_ds.dataset.transform = train_tfms
test_ds.dataset.transform  = test_tfms

In [28]:
BATCH_SIZE = 16

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.densenet169(pretrained=True)

for param in model.features.parameters():
    param.requires_grad = False

for param in model.features.denseblock3.parameters():
    param.requires_grad = True

for param in model.features.denseblock4.parameters():
    param.requires_grad = True

model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes)
)
model = model.to(device)

In [30]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam([
    {"params": model.features.denseblock3.parameters(), "lr": 5e-6},
    {"params": model.features.denseblock4.parameters(), "lr": 1e-5},
    {"params": model.classifier.parameters(), "lr": 1e-4},
])

In [31]:
labels = [label for _, label in dataset.samples]
class_counts = Counter(labels)

weights = [1.0 / class_counts[i] for i in range(num_classes)]
weights = torch.tensor(weights).to(device)

criterion = torch.nn.CrossEntropyLoss(weight=weights)


In [32]:
EPOCHS = 20

for epoch in range(EPOCHS):
    #TRAIN 
    model.train()
    train_loss = 0.0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Train"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    #VALIDATION 
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc="Validation"):
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    #Qtys
    val_f1_macro = f1_score(all_labels, all_preds, average="macro")
    val_f1_weighted = f1_score(all_labels, all_preds, average="weighted")

    val_accuracy = (torch.tensor(all_preds) == torch.tensor(all_labels)).float().mean().item() * 100

    print(
        f"Epoch [{epoch+1}/{EPOCHS}] | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val Acc: {val_accuracy:.2f}% | "
        f"F1 Macro: {val_f1_macro:.4f} | "
        f"F1 Weighted: {val_f1_weighted:.4f}"
    )